< 3월 13일 과제 >  
1. winequality-white.csv
2. 딥러닝 적용해보기
3. 모델 : LinearRegression - 회귀

In [125]:
# Data Load
import torch
import pandas as pd

wine_df = pd.read_csv('winequality-white.csv', sep=';')
wine_df.info() # data 깔끔

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4898 entries, 0 to 4897
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         4898 non-null   float64
 1   volatile acidity      4898 non-null   float64
 2   citric acid           4898 non-null   float64
 3   residual sugar        4898 non-null   float64
 4   chlorides             4898 non-null   float64
 5   free sulfur dioxide   4898 non-null   float64
 6   total sulfur dioxide  4898 non-null   float64
 7   density               4898 non-null   float64
 8   pH                    4898 non-null   float64
 9   sulphates             4898 non-null   float64
 10  alcohol               4898 non-null   float64
 11  quality               4898 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 459.3 KB


In [126]:
# 중복 확인
wine_df.duplicated().sum()   # 937
wine_df.drop_duplicates(inplace=True)

In [127]:
# 피처 지정
wine_df.corr().abs().quality.sort_values(ascending=False)

quality                 1.000000
alcohol                 0.462869
density                 0.337805
chlorides               0.217739
volatile acidity        0.190678
total sulfur dioxide    0.183356
fixed acidity           0.124636
pH                      0.123829
residual sugar          0.117339
sulphates               0.053200
free sulfur dioxide     0.010507
citric acid             0.007065
Name: quality, dtype: float64

In [128]:
import sys
sys.path.append('../')
from DL_class import scaler_model as sc

In [129]:
outliers = sc.find_outliers(wine_df)
outliers_flat = [item for sublist in outliers for item in sublist]
wine_df = wine_df.drop(outliers_flat, axis=0)

In [130]:
wine_df.reset_index(drop=True, inplace=True)
wine_df.tail(3)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
3128,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6
3129,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7
3130,6.0,0.21,0.38,0.8,0.020,22.0,98.0,0.98941,3.26,0.32,11.8,6


In [154]:
# Split
feature_one = wine_df['alcohol']
target = wine_df['quality']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(feature_one, target, test_size=0.2, random_state=11)

# Modeling
from sklearn.linear_model import LinearRegression

lin_model = LinearRegression()
lin_model.fit(X_train, y_train)
print('Test :', lin_model.score(X_test, y_test))

ValueError: Expected 2D array, got 1D array instead:
array=[11.4 11.1  9.7 ... 10.7 10.2 11.2].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [132]:
feature_one

,alcohol
0,9.5
1,10.1
2,9.9
3,9.6
4,11.0
...,...
3126,11.2
3127,9.6
3128,9.4
3129,12.8


<hr>

Deep Learning

In [133]:
import torch
import torch.nn as nn               # 인공신경망 층 관련 모듈
import torch.optim as optim         # 인공신경망 관련 함수
import torch.nn.functional as F     # 최적화 모듈

In [134]:
torch.manual_seed(718)      # 랜덤 시드 설정

In [135]:
# Transform train, test data as tensor type
X_train_ten = torch.tensor(X_train.values, dtype=torch.float32)
y_train_ten = torch.tensor(y_train.values, dtype=torch.float32)

##### Q. 근데 왜 굳이 split을 할까요? cost로 학습하는 거면 모두 넣어도 상관 없지 않을까요?

In [136]:
# 차원 추가
y_train_ten = y_train_ten.unsqueeze(1)

In [142]:
# target 수정 : 이진화
y_train_ten = torch.where(y_train_ten >= 6, torch.tensor([1.0]), torch.tensor([0.0]))

In [143]:
# shape, ndim, x_train 확인
print(X_train_ten.shape, X_train_ten.ndim, X_train_ten)
print(y_train_ten.shape, y_train_ten.ndim, y_train_ten)

torch.Size([2504, 1]) 2 tensor([[11.4000],
        [11.1000],
        [ 9.7000],
        ...,
        [10.7000],
        [10.2000],
        [11.2000]])
torch.Size([2504, 1]) 2 tensor([[1.],
        [1.],
        [0.],
        ...,
        [0.],
        [1.],
        [1.]])


2. 학습 준비

In [144]:
# Reset Weight, b value
W = torch.zeros(1, requires_grad=True)  
b = torch.zeros(1, requires_grad=True)
print(f'W : {W}, b : {b}')
# requires_grad= : 학습을 통해 값이 계속 변하는 변수임을 의미
# (If autograd should record operations on the returned tensor.)

W : tensor([0.], requires_grad=True), b : tensor([0.], requires_grad=True)


In [151]:
# Set SGD : Updata W, b
# - W, b, learning rate
optimizer = optim.SGD([W, b], lr=0.0001)

3. 학습 진행

In [152]:
epochs = 100

for epoch in range(epochs+1):
    # print(epoch)
    # 1. function
    y_pred = X_train_ten * W + b
    
    # cost
    cost = torch.mean((y_pred - y_train_ten)**2)
    
    # Upgrade H(x) by cost
    optimizer.zero_grad()        # gradient 초기화
    cost.backward()              # cost 미분
    optimizer.step()             # Update W, b
    # print(f'W : {W.grad}, b : {b.grad}')
    
    
    if epoch % 5 == 0:
        print(f'Cost: {cost.item()}')

print(W, b, cost, sep='\n\n')

Cost: 6060.40673828125
Cost: 6060.0517578125
Cost: 6059.7412109375
Cost: 6059.4658203125
Cost: 6059.2177734375
Cost: 6058.98974609375
Cost: 6058.78173828125
Cost: 6058.5869140625
Cost: 6058.4033203125
Cost: 6058.2265625
Cost: 6058.05615234375
Cost: 6057.88818359375
Cost: 6057.73046875
Cost: 6057.576171875
Cost: 6057.42431640625
Cost: 6057.2744140625
Cost: 6057.12548828125
Cost: 6056.9775390625
Cost: 6056.83056640625
Cost: 6056.68359375
Cost: 6056.53759765625
tensor([-66.1156], requires_grad=True)

tensor([711.5611], requires_grad=True)

tensor(6056.5376, grad_fn=<MeanBackward0>)


4. 결과 확인

In [153]:
pred_tensor = torch.tensor([[9.0]], dtype=torch.float32)
predic = pred_tensor.matmul(W) + b
predic.item()

116.5211181640625

Cost 의 값이 무한대로 커지면서, 예측값도 너무 커진다.
y학습값의 차원을 x학습과 동일하도록 수정했는데도 동일한 문제가 발생한다.

우선 Logistic Reg 파일을 따라 진행해볼 것이다.